## Setup


[Setup Python Virtual Environment in VSCode](/venv_setup.md)


**Security Note:** Using fixed package versions ensures stability and reproducibility, but may expose you to security vulnerabilities if not regularly updated. Balance stability and security by periodically reviewing and updating packages.

In [ ]:
# Install required packages in a virtual environment (Currently only tested with Python 3.12)
%pip install --upgrade pip
%pip install pandas==2.2.*
%pip install requests==2.32.*
%pip install JLC2KiCadLib==1.0.32

# Download JLCPCB Parts CSV File 

In [29]:
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url}/{filename} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Download the file
download_file(url, "jlcpcb-components-basic-preferred.csv")

Deleted existing file: jlcpcb-components-basic-preferred.csv
Downloaded https://cdfer.github.io/jlcpcb-parts-database/jlcpcb-components-basic-preferred.csv to jlcpcb-components-basic-preferred.csv


ToDo:

Create Symbols/Footprints for:
20617924
20617927
28646267

3rd Pin on Inductor: C2858862

Ferrite Bead Symbol Value

Cleanup .step files

Merge in display driver symbol lib

Add TVS diodes to Mosfet Footprints

Extended Components
Addressable LED (low cost, rgbW)
ESP32 Chips (OG,S2,S3,C3)
40Mhz Crystal
RP2040
USB C Connectors (Low cost, full 24P, )
Encoder (alps?)
GPS Module
GPS patch antenna (25x25?)
Humidity Module
Motor Driver
Display (ssd1306?)
RiscV (CH32V003?)
USB Hub IC
Cellular (SIM7080G)
Relay
Accelerometer
Stepper Driver
USB-Serial (CH340?)
VEML7700
TCS34725
ATGM332D-5N31

In [90]:
import JLC2KiCadLib
import JLC2KiCadLib.JLC2KiCadLib as JLC
import os

componentList = [
2841497
]

os.chdir("..")

class Args:
    def __init__(self):
        self.footprint_creation = True
        self.symbol_creation = False
        self.footprint_lib = "JLCPCB-Kicad-Footprints"
        self.output_dir = "JLCPCB-Kicad-Library"
        self.model_base_variable = ""
        self.model_dir = "3dModels"
        self.models = ["STEP"]
        self.skip_existing = True
        self.symbol_lib = "JLCPCB-Diode-Packages"
        self.symbol_lib_dir = "JLCPCB-Kicad-Symbols"


args = Args()

for component in componentList:
    JLC.add_component(f"C{component}", args)

os.chdir("JLCPCB-Kicad-Library")

## Cleanup footprints

In [ ]:
import re
import os

def read_kicad_mod_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines


def write_kicad_mod_file(filename, lines):
    with open(filename, 'w') as file:
        file.writelines(lines)


def round_coordinates(lines):
    for i, line in enumerate(lines):
        if line.startswith('\t\t(at ') or line.startswith('\t\t(start ') or line.startswith('\t\t(mid ') or line.startswith('\t\t(end ') or line.startswith('\t\t\t(width ') or line.startswith('\t\t(size ')  or line.startswith('\t\t(center '):
            parts = re.findall(r'-?\d+(?:\.\d+)?', line)
            rounded_parts = []
            
            for part in parts:
                if '.' in part:
                    rounded_part = f"{float(part):.2f}".rstrip('0').rstrip('.')
                    if rounded_part == '-0': rounded_part = '0'
                else:
                    rounded_part = part
                rounded_parts.append(rounded_part)
            
            rounded_line = line
            for old, new in zip(parts, rounded_parts):
                rounded_line = rounded_line.replace(old, new)
            
            lines[i] = rounded_line
            if line != rounded_line:
                print(f'{repr(rounded_line)} <= {repr(line)}')
    return lines

# Example usage:
footprint_names = [os.path.splitext(filename)[0] 
                for filename in os.listdir('JLCPCB-Kicad-Footprints') 
                if filename.endswith('.kicad_mod')]

for filename in footprint_names:
    path = f"{os.path.join('JLCPCB-Kicad-Footprints', filename)}.kicad_mod"
    footprint_lines = read_kicad_mod_file(path)
    rounded_lines = round_coordinates(footprint_lines)
    write_kicad_mod_file(path, rounded_lines)

In [ ]:
import re

def read_kicad_mod_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines


def write_kicad_mod_file(filename, lines):
    with open(filename, 'w') as file:
        file.writelines(lines)


def set_silkscreen_width(lines, width=0.15):
    new_lines = []
    updating_width = False
    width_line_index = 0

    for i, line in enumerate(lines):
        # Debug print: Show the current line and its index
        # print(f"Line {i}: {repr(line)}")
        
        # Check if the line starts an line element
        if re.search(r'\s*(fp_line|fp_arc|fp_circle)', line):
            updating_width = True
            print(f"  Detected an element: {repr(line)}")
        
        # Check if the line starts the stroke property
        elif updating_width and re.search(r'\s*stroke', line):
            print(f"  Detected stroke property: {repr(line)}")
        
        # Check if the line defines the width
        elif updating_width and re.search(r'\s*width', line):
            # Debug print: Show the line before replacement
            print(f"  Found width property on Line {i}: {repr(line)}")
            width_line_index=i
        
        # If the line ends a silkscreen element, stop updating width
        elif re.search(r'\s*\(layer "F.SilkS"\)', line) and updating_width:
            # Replace the width value with the new one
            new_lines[width_line_index] = re.sub(r'\s*width\s+[0-9.]+\s*', f'width {width}', new_lines[width_line_index])
            # Debug print: Show the line after replacement
            print(f"  Updated width property: {repr(new_lines[width_line_index])}")
            updating_width = False
        
        # Append the line to new_lines
        new_lines.append(line)

    return new_lines



# Example usage:
footprint_names = [os.path.splitext(filename)[0] 
                for filename in os.listdir('JLCPCB-Kicad-Footprints') 
                if filename.endswith('.kicad_mod')]

for filename in footprint_names:
    path = f"{os.path.join('JLCPCB-Kicad-Footprints', filename)}.kicad_mod"
    footprint_lines = read_kicad_mod_file(path)
    fixed_lines = set_silkscreen_width(footprint_lines, width=0.15)
    write_kicad_mod_file(path, fixed_lines)
